In [1]:
from pathlib import Path
import shutil
import math
import random

We will only use classes with at least three images

In [2]:
# Data is one directory up
base_path = Path('.').absolute().parent
# Path to raw motorcycle images
raw_path = base_path / 'data/motorcycles'
# Path to images that use padding to make them square
square_path = base_path / 'data/square_motorcycles'

In [3]:
# Get the image counts
raw_target_paths = [path for path in raw_path.iterdir()]

target_counts = []
for target_path in raw_target_paths:
    image_paths = target_path.iterdir()
    num_images = sum([1 for image in target_path.iterdir()])
    target_counts.append((target_path.name, num_images))


In [4]:
def get_image_counts(target_path):
    # Counts the number of images in each subdirectory of a target path
    # target_path: pathlib.Path object of the base path holding the subdirectories
    # Returns a list of tuples with (target, image_count)
    target_paths = [path for path in target_path.iterdir()]
    image_counts = []
    for target_path in target_paths:
        image_paths = target_path.iterdir()
        # Might be tricky to read. The list comprehension just returns 1 for each image
        # Summing these up provides the count
        num_images = sum([1 for image in target_path.iterdir()])
        image_counts.append((target_path, num_images))
    return image_counts

In [5]:
def get_minimum_images(target_paths, min_images):
    return [count for count in target_paths if count[1] >= min_images  ]

def get_maximum_images(target_paths, max_images):
    return [count for count in target_paths if count[1] <= max_images  ]

In [6]:



# Get the raw counts, keeping only targets with at least three images
raw_counts = get_image_counts(raw_path)
raw_enough = get_minimum_images(raw_counts, 3)
# Get the square counts, keeping only targets with at least three images
square_counts = get_image_counts(square_path)
square_enough = get_minimum_images(square_counts, 3)

At this point, we have paths to the image directories we believe have enough data to work. We have also processed these images (or attempted to), so I believe they are clean enough to use. We will copy them to a new directory with separation between train, val and test, which will hold all the data we will use. 

In [7]:
def segment_images(target_paths, test_proportion=0.2, val_proportion=0.2):
    '''
    Segments images into training, validation and test, with some sensible defaults
    target_paths: A list of tuples with (target path, a count of images in that path)
    '''
    segmented_paths = []
    for path in target_paths:
        segments = {}
        # Take the proportion rounded up. (This might give us fewer training images, but we will use augmentation in training)
        num_test = math.ceil(path[1] * test_proportion)
        num_val = math.ceil(path[1] * val_proportion)
        # Take the rest for train 
        num_train = path[1] - num_test - num_val    
        # Shuffle the images to make sure we do not have an order we didn't think of. It's possible that the images
        # are sorted by site or something like that.
        images = [image for image in path[0].iterdir()]
        random.shuffle(images)
        segments['test'] = images[0:num_test]
        segments['validation'] = images[num_test: num_test + num_val]
        segments['train'] = images[num_test + num_val:]
        segments['total_images'] = path[1]
        segments['target'] = path[0].name
        segmented_paths.append(segments)
    return(segmented_paths)



In [8]:
segmented_raw_paths = segment_images(raw_enough)

segmented_square_paths = segment_images(square_enough)

In [9]:
def copy_clean_data(segmented_paths, clean_subdir):
    ''' 
    Copies images from subdirectories into ./clean_data/[train, val, test]
    target_paths: list of dicts witht he paths to images
    clean_path: String with the name of the directory inside clean_data to use as the base path
    '''
    # Create needed top-level directories
    clean_base_path = base_path / ('clean_data/' + clean_subdir)
    clean_base_path.mkdir(exist_ok=True, parents=True)
    clean_train_path = clean_base_path / 'train'
    clean_train_path.mkdir(exist_ok=True, parents=True)
    clean_val_path = clean_base_path / 'val'
    clean_val_path.mkdir(exist_ok=True, parents=True)
    clean_test_path = clean_base_path / 'test'
    clean_test_path.mkdir(exist_ok=True, parents=True)
    
    # Cycle through the segmented data, copying to train, val and test directories
    for target in segmented_paths:
                # Handle training data
        train_path = clean_train_path / target['target']
        train_path.mkdir(exist_ok=True, parents=True)
        for path in target['train']:
            # Long file names can cause problems
            suffix = '.jpg'
            if len(path.suffixes) > 0:
                suffix = path.suffixes[0]
            file_name = path.name[:50] + suffix
            print(file_name)
            shutil.copy(str(path), str(train_path) + '/' + file_name)

        # Handle validation data
        val_path = clean_val_path / target['target']
        val_path.mkdir(exist_ok=True, parents=True)
        for path in target['validation']:
            # Long file names can cause problems
            suffix = '.jpg'
            if len(path.suffixes) > 0:
                suffix = path.suffixes[0]
            file_name = path.name[:50] + suffix
            shutil.copy(str(path), str(val_path) + '/' + file_name)

        # Handle test data
        test_path = clean_test_path / target['target']
        test_path.mkdir(exist_ok=True, parents=True)
        for path in target['test']:
            # Long file names can cause problems
            suffix = '.jpg'
            if len(path.suffixes) > 0:
                suffix = path.suffixes[0]
            file_name = path.name[:50] + suffix
            shutil.copy(str(path), str(test_path) + '/' + file_name)
        




In [ ]:
copy_clean_data(segmented_raw_paths, 'motorcycles')
copy_clean_data(segmented_square_paths, 'square_motorcycles')

## Get targets where there are at least eight per class

The model was not generalizing well, so we want to test out what happens when we only use targets with at least eight images. 

In [46]:
eight_min_data = get_minimum_images(raw_counts, 8)
segments = segment_images(eight_min_data)
copy_clean_data(segments, 'eight_min')

e2b0f10f-680a-4c7b-95de-8d69396f163c-INDIAN-Chief-.jpg
877f1113-674f-4a69-8ffe-52f26a3ac1a2-2010-Indian-C.jpg
ca62efbd-0101-4a34-bd17-66b6fec41564-2010-Indian-C.jpg
c69d94b6-0aab-44c4-be5f-c32e20ae1ce6-2010-indian-c.JPG
4a90efee-c063-455c-826b-6e1c40953bb1-indian-chief-.jpeg
9ab0798c-f966-48c3-aa54-f7c20f56b152-2009-suzuki-b.jpg
4c94c656-1d66-48fa-a1cc-15717ee62661-2009-suzuki-b.jpg
3d49b025-0a3c-4128-aae0-ae5ac786142e-2009-suzuki-b.jpg
acb70764-3a6b-475c-affd-fa6f42bbc732-SUZUKI-Boulev.jpg
bf78d0de-a90e-4db2-af96-bac130a37747-2016-boulevar.jpg
3e8a1441-d82f-4a91-90f4-67e89b8a288b-2009-suzuki-b.jpg
31cbed24-7adc-4579-92f0-3f3447146f26-2011-Triumph-.jpg
d43e7cfd-4d93-4c26-8b3e-5ec7673deb21-triumph-ameri.jpeg
0cc1e766-5485-4b9b-b663-4ee93830c85a-triumph-ameri.jpeg
457a75c7-f25d-496b-9625-b90ba8c78ea2-2011-Triumph-.jpg
95ba873f-d323-4705-bdf5-9415577fcb77-301012_2013_i.jpg
df45ec67-9a2f-49ad-93b2-80e4310bc4de-indian-chief-.jpg
86939356-3809-4a75-87ea-870c16103178-indian-chief-.jpg
8f6d0eb

b5326bbc-3ac6-4f13-ae82-6cbbc271bfb6-2017-Ducati-S.jpg
07d28423-0807-443a-a216-678ff65a6156-12240331_1015.jpg
d8b9a289-5171-4e69-a7b8-977002b4fb85-2017-Ducati-S.jpg
5a9181e1-abdf-40e4-b5c4-b83137a050da-022017-Ducati.jpg
38f9aa48-b064-4b04-9e05-0808d2bfd932-2017-Ducati-S.jpg
013b8298-7afd-4960-b8d2-681ca4d092c3-2017-Ducati-S.jpg
bd9f0448-4b01-4991-8174-121c04668c29-TTL_7627-Edit.jpg
3cee831d-2fd6-4359-9e17-95f22c7dd693-2017-xdiavel-.jpg
d6d54d79-0320-45c0-992a-b99eb14d9b2d-TTL_7630-Edit.jpg
afb97f7c-7afc-409b-bb11-618004c1192a-TTL_7640-Edit.jpg
42410f80-60a7-47b5-9532-00d1f97b4950-Ducati-XDIAVE.jpg
692cddb5-2e4e-43bf-9033-1c28ab5a792a-095e23cc-11c0.jpeg
2f901eb3-652f-4879-bdca-18b326ca7d65-2017-Ducati-X.jpg
89b94fe9-0522-4aff-bb79-76b61486a98c-2017-xdiavel-.jpg
8d521954-1073-42f0-a0de-1eaca42e534f-TTL_7634-Edit.jpg
b71d3b6b-3cc0-495e-a76f-953e37abd7bc-2000000001.jp.jpg
8413a8b1-7932-415d-91d8-ca7e57fb2095-6000000001.jp.jpg
63c8020a-e5c5-4629-a0a3-621b34670d87-harley-CVO-18.jpg
ab99106c-

15aea54a-58f1-4008-ba89-80a83de8c83d-2017_CRF125F_.jpg
d2fa18c4-5d9f-45a1-9416-001f8b74cb49-CRF125F_2017_.jpg
30c98481-204f-4265-8282-8e753c939253-2017-Honda-CR.jpg
af69df89-b3a8-4f89-8da6-09a9c2d77d5e-2017-Honda-CR.jpg
d199f29e-e007-44ee-a558-e4755b1301e0-maxresdefault.jpg
4cbe01d1-418f-41b1-a9c2-a07f664b1c96-5166279_04578.JPG
1b31725a-3c13-42c9-9538-d3f42a59c3e2-honda-crf125f.jpg
cbd6b089-37ae-4a2e-8720-3064b57ad1c1-2017-Honda-CR.jpg
dc8cf826-6080-4c6a-8eb4-1f8da5d60a13-2017-Honda-CR.jpg
37ff0bb8-fd7d-4248-9043-4428c5ea79f9-5166279_00069.JPG
12dc0ee1-2331-4143-9b10-68b67521b979-crf125fb_1306.png
9fb7038a-cab3-4abc-ad56-1876003db198-6000000002.jp.jpg
3529aa7d-250e-4ad2-8766-7ab7318ff034-2016-Honda-CR.jpg
929f9964-d2a3-43f9-a8cd-13895e854802-2017_honda_cr.jpg
f4c8b095-b714-4edf-ab1c-200b1dff0395-2017_honda_cr.jpg
70760aa2-7564-4ad3-a138-3eced547d810-2017_honda_cr.jpg
aef33a41-68a8-4161-b891-f8e3763a6e98-2016-Honda-CR.jpg
ef53e1b5-9323-4e40-9044-75f164cfab98-maxresdefault.jpg
01fb391c-2

98980602-42bc-4061-b06f-6574b82ff067-2016-klr-650-.jpg
60dc1f62-0403-43e5-8d14-093099c187fd-2017-Kawasaki.jpg
b9f642a5-33dc-4bff-a05e-e205898a667e-KLR650-First-.jpg
11a4a32b-b391-48c8-9907-241d5fca9124-a1bigkup.agx..agx
1d08d648-e06f-4542-a25e-ff59e2c1d68f-2017-Kawasaki.jpg
d43f9e20-2250-46e1-9f4e-ca0b07253fc8-KLR650-Left-S.jpg
c91bbb6d-f96a-4337-a983-8dbb17d8e00c-6000000001.jp.jpg
c8ad3088-46f5-4b28-911f-f91707b80d31-2017-Kawasaki.jpg
8bf1e57f-d15e-4714-9e57-9ca2cc575b6c-KAWASAKI-KLX-.jpg
b7071636-ea9b-4ecd-a986-063183e5ac92-12FC7199-5C4D.jpg
92780a2e-8e4b-4547-b1d4-212684916e6a-2017-Kawasaki.jpg
f084642d-a2aa-44a2-9c6c-da74c3b3dec8-6000000005.jp.jpg
95bb3c96-7326-4ffb-b70b-751820c7507e-qf0vzwng.ihn..ihn
ca069adf-1718-4ae8-9fe2-774693f99aab-6000000002.jp.jpg
a25fb71e-7b6f-4ee1-bb74-8f1fbbc5634d-6000000002.jp.jpg
c75b0ae3-a425-4125-85e7-3d8f2514bc85-6000000003.jp.jpg
7c39fa5d-fa5d-40aa-8afe-5f4dd276b55c-6000000003.jp.jpg
ae5252f3-ea3a-4462-af85-0b6f7c7add11-6000000001.jp.jpg
5bb304c4-e

bee05b36-6bf4-426e-850f-e4741d4104af-2017-Suzuki-B.jpg
36eb7956-dc17-4578-b45f-89467d959225-2017-Suzuki-B.jpg
2ea8908e-6f30-484d-8cc3-789cac8359bb-2017-Suzuki-B.jpg
7fc2bc16-324e-4ad2-978f-a0621dc908c5-1d5329fbeac41.jpg
b35ff62b-2ff4-4e4b-8874-250b74bf18ea-596a836ddd4c1.jpg
d303e464-33c3-437d-81d0-19fa8e3aa296-2017-Suzuki-B.jpg
8174d9e2-1b4f-477d-8244-9bdcbf26a627-2017-suzuki-b.png
f7ab5738-74b7-481f-9dcb-ac9ac13e656c-6000000002.jp.jpg
f6179af4-0fd9-4334-b365-908eb13d7181-2017-Suzuki-B.jpg
e7616427-1629-41ab-9619-50a6c6e612c1-maxresdefault.jpg
08e4b1b0-b624-41b8-ae48-7a6bdfa4b103-4901851-0-185.jpg
51fdd928-8b73-498e-adbd-0868ef3db035-maxresdefault.jpg
52333c82-59c6-4ffd-a9e8-9f57f833ae30-maxresdefault.jpg
b9fc50d7-d29d-465b-bc3e-2b2abb715354-6000000003.jp.jpg
c6808f07-a78a-4a8e-b411-70223f5ab81b-img_jeDLiULuI.jpg
37b6114e-3644-4094-912f-ee1eb4c3a34e-2017-Suzuki-D.jpg
64e2cac9-0dc8-400d-91a4-8ad2601e0a46-2016-dr200s_1.jpg
411cab61-d487-403a-b468-4813233b84e6-6000000001.jp.jpg
04409eb9-4

f4897fdb-aa46-4a3e-8bbe-09af468ccf60-2017-FZ6R-Yam.jpg
e720d42b-8e6a-433b-94db-caddc5a7a7a1-6000000001.jp.jpg
7476c0f3-d1a0-4453-8814-6f9209e06db3-2016-yamaha-f.jpg
6bd017c3-9583-46e3-adea-71760d254418-2017-FZ6R-Yam.jpg
8420f5ff-ff7a-45fe-aef4-5aafd0983f13-2017-yamaha-f.jpg
8ce5ef85-c764-4eb7-8b95-071d8328af13-2016-yamaha-f.jpg
2ac7d81d-ba04-4101-80d9-1e1b93814e37-2017-Yamaha-F.jpg
b3b10ef5-a5e1-4dcb-8de5-f62a7b3266b0-2017-Yamaha-F.jpg
9ae0161d-1f4e-4f3f-90ae-d3aba2ef5cfe-2017-FZ-07-Ya.jpg
5b003f36-7f17-406f-be43-1726c9d9c23d-2017_FZ-07_bl.jpg
c9ecdadb-79bb-4429-94fa-b94fb61bf53c-2000000003.jp.jpg
897376c8-5ff6-46ad-9531-6984a7cc3ebf-2017-FZ-07-Ya.jpg
d52fb0df-41b9-4419-834f-3e1da6d8e889-2000000013.jp.jpg
177c558d-5de1-46b0-b8be-4e585bb03446-2017-Yamaha-P.jpg
f7d881ee-4410-4f4b-9ec8-7b695c8ed0e8-6000000004.jp.jpg
9f19f22e-9e12-460c-97f5-bfac3f30034b-yamaha-pw50.j.jpg
bfcd64a8-ae57-49c6-9435-80bdec6ebf6d-2017-Yamaha-P.jpg
e373518d-3f35-4f57-abe4-ecddaf62733d-2017-Yamaha-P.jpg
79bd4f43-9

461b4e77-4620-4956-bd9f-1aaaa1522d25-2018-Harley-D.jpg
34717443-b9d8-4f1b-89d5-dc70344619f0-2018-Harley-D.jpg
788d986a-4361-49f3-b871-147ad77002a7-2018-Harley-D.jpg
3816352e-9ba1-4e03-a9c4-d6fd8f7e9660-2018-Harley-D.jpg
69d12fa8-095b-4d58-841b-7ae4d3a0e02d-2000000002.jp.jpg
e8a98df6-ed59-46be-bda6-2f7c13274342-2000000001.jp.jpg
5f1e6ae7-e4e5-4377-b85c-e98284100ee7-2018-Harley-D.jpg
39deb164-57d2-489c-96f4-fc0a8111cac5-2000000003.jp.jpg
32da0995-95b2-4058-8f73-91d33bb1339b-s-l300.jpg.jpg
6c247156-7ab6-4141-bfc4-3c0e4c373ba5-2018-Honda-NC.jpg
ad3c3f6b-0953-44a3-9eb5-e6aebe98e882-2018-Honda-NC.jpg
e4432135-85bc-476d-b9c6-d6bcf018a215-9Honda%20NC75.jpg
a673d282-58c6-4dab-ac12-557497e6696b-maxresdefault.jpg
8c40a4f4-cd55-45db-b6bc-c2431d2ef754-maxresdefault.jpg
aa4bc6db-dea0-4f5a-a30b-e779f1a68d3e-2018-Indian-C.jpg
7d8eea43-0710-401f-a88f-4b4df7bc5401-2018-Indian-C.jpg
73147aa4-1bd0-4b82-b07d-d3ea538c6459-2018-Indian-C.jpg
3abbe139-9432-4b99-929a-b067ee178b9d-2018-Indian-C.jpg
a75325e7-d5e2

ca3fa408-da14-4636-837b-aa43c304397e-6000000009.jp.jpg
42bd3fc3-6c1d-4a73-9d6e-2b30a85a0288-6000000009.jp.jpg
b6124704-e913-4a85-857b-48c269710022-e14b44d0-f083.jpeg
e83c0635-3bfb-421c-9d09-edc2f8af3116-6000000001.jp.jpg
2000e6fa-2a8e-4b18-a01b-8b50965e25df-maxresdefault.jpg
ce750e8a-a264-4d15-ad4b-a93d510f7b17-2.jpg.jpg
8a977e62-4c1b-4bfc-a3ca-af5189fbd41b-maxresdefault.jpg
54b7da4d-64f9-4f25-9b08-bb24847263d1-4.jpg.jpg
0b1e9da0-3d40-4594-a299-252cdcb6f179-8.jpg.jpg
e0a9d443-e471-4a3e-9786-db29c95801ea-11.jpg.jpg
23ffed40-e7e2-4a27-b1be-7699a4e0c534-4.jpg.jpg
121c9300-2bfa-47f3-81ed-ce10dbbe0dfd-thumb2-triump.jpg
b2043638-399e-4317-9b56-a5cbca5891f1-2018-Triumph-.jpg
7c15063f-886d-4853-8a34-fc31db59f70b-2.jpg.jpg
4c3aa87b-c315-4d3e-af44-dd4c6bae80f7-9.jpg.jpg
7f4e09d1-5b2b-4e20-b530-149c027fb321-2018-Triumph-.jpg
b6bb668e-63e8-4a19-bcfc-8e1177924a64-Triumph-Thund.jpg
ff13141c-7ee6-4c88-a65a-1aa6c4e33436-2018-Triumph-.jpg
7df86d24-ab2b-4bdb-8960-01adaf9ed039-hqdefault.jpg.jpg
1013f4da-

## Make a dataset with more balance (move more images to val and test)

In [47]:
# Balance between train, val and test
eight_min_data = get_minimum_images(raw_counts, 8)
segments_balanced = segment_images(eight_min_data, test_proportion=0.3, val_proportion=0.3)
copy_clean_data(segments_balanced, 'eight_min_balanced')


1cbac4ca-25bd-4843-9d3e-2cedaf3195d0-indian-chief-.jpeg
c69d94b6-0aab-44c4-be5f-c32e20ae1ce6-2010-indian-c.JPG
4a90efee-c063-455c-826b-6e1c40953bb1-indian-chief-.jpeg
c6bafaab-4b8d-4b93-b00e-096ab41b0fce-SUZUKI-Boulev.jpg
8307b47e-9628-424e-9d2f-cbce01f26694-2010-suzuki-b.jpg
9c8fee27-965e-4303-b156-7d5592595dba-2009-suzuki-b.jpg
acb70764-3a6b-475c-affd-fa6f42bbc732-SUZUKI-Boulev.jpg
31cbed24-7adc-4579-92f0-3f3447146f26-2011-Triumph-.jpg
0cc1e766-5485-4b9b-b663-4ee93830c85a-triumph-ameri.jpeg
33970764-8ade-4ff7-af62-09b163679b8f-indian-chief-.jpg
e0a48fd8-82f7-456a-bd8f-416b51572308-2013-indian-c.jpg
95ba873f-d323-4705-bdf5-9415577fcb77-301012_2013_i.jpg
a13aa685-bd19-43ff-a587-7ea84663add0-301012_2013_i.jpg
e5eb168a-9567-4146-bc61-c6bad6a60850-moto-guzzi-gr.jpg
1bf08c8e-2696-4df9-abc4-81ae719f3240-Moto-Guzzi-20.jpg
bca95a81-527f-4230-abfb-e08953e0f97c-2015-moto-guz.jpg
76f2bcd2-27dd-4c3f-8f72-95cb8e0a7099-Moto-Guzzi_Gr.jpg
c00d4632-bd52-40df-8487-6f8ca17394ef-DSC05595_1485.jpg
2ba8952

2f14ae49-2d3f-4557-b76f-8be9ad49a23b-Harley-Davids.jpg
0f4dfea2-5c36-42e7-9311-2b7368bdae8b-Harley-Davids.jpg
57b03d30-cf6a-4869-bfe4-de5cb817ae0a-Harley-Davids.jpg
3666ae38-33cf-4b03-91ed-50efc9e156a2-2000000040.jp.jpg
0db4ce9d-7670-4553-ad44-f368ec1e13ef-Harley-Davids.jpg
36670b65-6525-4374-b1bc-a5eda7f245c0-2017-harley-d.jpg
552da750-d8b7-4cf7-89b9-78ff7b3a196e-030917-2018-h.jpg
62a6e85d-7639-4b7e-ab0e-6ccdffd869a3-2017-Sporster.jpg
5529b423-5e56-4ead-beff-6057ca6a23f7-harley-davids.jpg
0ba7187e-e9be-4f67-aa96-971d3170547e-Harley-Davids.jpg
c4a666c7-0a9a-400b-9440-1aea5b2255d2-2000000002.jp.jpg
5f424fb8-e859-46a7-a6d0-883069f3499f-2018-Harley-D.jpg
5b24ddc4-2fe8-4d5e-b2c6-ade1c7856403-E0A4E698-96F7.jpg
f8144458-4f5c-41ba-bf65-2fb636f21a8e-Harley-Davids.jpg
8066d722-8474-4501-b997-2eaa3581eb39-LIMITED-LOW.j.jpg
f6954aab-d5a0-4f1e-b2c0-a1e956e62de8-2016-Ultra-Li.png
2323f7fd-ba01-4562-b7e4-dc275d44080c-16-hd-electra.jpg
6f49c455-1488-4b2a-bf55-1d5e726c8a73-2016_h-d-v-ro.jpg
cc4459db-5

c6ad2838-0848-4f7c-82f5-1a4f2d42b576-6000000002.jp.jpg
7325234e-1fef-45a8-8574-23498e6df075-2017_Kawasaki.jpg
6d9333ab-03d0-443f-a9ec-6a64a3f7aecc-kawasaki-ex-2.jpg
69625efd-41b9-4d23-b3a5-e6015e99d5e8-6000000001.jp.jpg
452d4b0a-7b53-42db-9a9e-ea97566defd9-2017-Kawasaki.jpg
a4df6f7b-793f-4341-ab4f-f50a27d0b29c-2017-ninja-h2.jpg
d4fd821b-a43f-46ca-b074-d71140b37846-KAWASAKI-Ninj.jpg
4f69980a-40e7-4519-a919-e86f0c09f9de-maxresdefault.jpg
aae06d81-866e-4d29-b292-5e849f46e115-Kawasaki_Ninj.jpg
b666f3ec-9fd5-47f0-b542-0ae8fea41ccb-2000000012.jp.jpg
12579e04-4614-4244-8d53-59a54082e622-kawasaki-2017.jpg
e1f05024-a975-4f40-a852-75a7d84deb53-2017-Ninja-ZX.jpg
afcee251-b7b5-44e8-84af-526b34e726ed-2017-Kawasaki.jpg
480b9126-febf-4feb-9771-1b186b6c143e-2017-Ninja-ZX.jpg
856b8d52-ca9c-4130-9dbc-2bd4741e1626-2017-Ninja-ZX.jpg
2220f338-be1e-4d3e-adc7-54c21f8c1f60-maxresdefault.jpg
fc2f85fa-4e30-445a-b6b1-b6a242c785b9-4nbhasbo.0ci..0ci
6ca78d29-c752-47ff-880a-856b0d97f9c4-BFBB304C-57FC.jpg
15ffdb0a-2

6ee4cc1a-ae7d-4d75-bf40-a05e42aea097-2018-Suzuki-B.jpg
d756a283-63a0-41b5-b3d8-9cc069253c44-2018_Suzuki_B.jpg
52260968-1b03-4b86-bc5f-a850addc410b-Suzuki-Burgma.jpg
8f70ab9b-1d2f-4927-94e8-eee1ca5c346c-2018-Suzuki-B.jpg
f594d217-6b5d-44a6-9cb0-a908662fc8d4-4507EAA1-EBFC.jpg
0f4f88ba-daee-4f79-a588-a5a4463b00a1-2018-Suzuki-G.jpg
27bc7f76-7b95-41b8-8ff5-74b8753be391-2018-Suzuki-G.jpg
2024de3f-4e87-4a31-a4de-ad0fdd4fbbb1-2018-Suzuki-G.jpg
814b4069-a9a5-428a-9f8c-a112212ecd33-6000000001.jp.jpg
d8b5ca1d-14a1-4471-bc4a-897b509b2913-112117bottom..jpg
ebf555ea-8c44-4a2b-beae-0f096544750d-2018-Suzuki-S.jpg
59f8ea11-e44d-4f84-b960-692823a1dd49-2018-Suzuki-S.jpg
1acb208b-4c13-4812-925e-a451c3d927ef-6000000001.jp.jpg
9c56bc1b-6173-4196-a8e2-21c1689b421d-SUZUKI-VANVAN.jpg
2e7aec85-545a-4afe-af60-48fb04a6a384-maxresdefault.jpg
899bbe61-716d-49bb-b078-ab2c533a9a74-2018-V-Strom-.jpg
94ceeba2-0c4a-4a23-b9b7-f0574f170065-v-strom%20100.jpg
1286b620-b51c-4a34-9553-9555ab263796-2018-Suzuki-V.jpg
2312cc3a-4

## Make a dataset with fewer train and test

In [48]:
eight_min_data = get_minimum_images(raw_counts, 8)
segments_nonbalanced = segment_images(eight_min_data, test_proportion=0.1, val_proportion=0.1)
copy_clean_data(segments_nonbalanced, 'eight_min_nonbalanced')


a4b222ed-ccf0-4458-a954-7440dab4ffb2-2010-Indian-C.jpg
4a90efee-c063-455c-826b-6e1c40953bb1-indian-chief-.jpeg
1cbac4ca-25bd-4843-9d3e-2cedaf3195d0-indian-chief-.jpeg
3e664308-bb7e-43d6-b335-846d85a03cb3-2010-Indian-C.jpg
fa203b8b-d93c-4c51-a4f2-f209aef44df4-2010-Indian-C.jpg
c69d94b6-0aab-44c4-be5f-c32e20ae1ce6-2010-indian-c.JPG
ca62efbd-0101-4a34-bd17-66b6fec41564-2010-Indian-C.jpg
404ed66d-87ba-4f79-ac69-71ac01e4a058-2009-suzuki-b.jpg
acb70764-3a6b-475c-affd-fa6f42bbc732-SUZUKI-Boulev.jpg
3e8a1441-d82f-4a91-90f4-67e89b8a288b-2009-suzuki-b.jpg
9ab0798c-f966-48c3-aa54-f7c20f56b152-2009-suzuki-b.jpg
9c8fee27-965e-4303-b156-7d5592595dba-2009-suzuki-b.jpg
3d49b025-0a3c-4128-aae0-ae5ac786142e-2009-suzuki-b.jpg
bf78d0de-a90e-4db2-af96-bac130a37747-2016-boulevar.jpg
8307b47e-9628-424e-9d2f-cbce01f26694-2010-suzuki-b.jpg
d43e7cfd-4d93-4c26-8b3e-5ec7673deb21-triumph-ameri.jpeg
457a75c7-f25d-496b-9625-b90ba8c78ea2-2011-Triumph-.jpg
31cbed24-7adc-4579-92f0-3f3447146f26-2011-Triumph-.jpg
2ecf4d6

47a09179-d0bb-40ff-b579-782839987aad-2017-Ducati-D.jpg
172dfc9c-0fab-45fb-af1f-027da38d42ec-2018-Ducati-H.jpg
d1b3547d-da96-467e-8e71-d6799adfbd25-2017_Ducati_H.jpg
fec36a88-7a3d-4b97-86ff-1f55b04a114b-27-Hypermotar.jpg
c227100f-8a1d-46b4-8880-00c5cc568b88-2016-Ducati-H.jpg
6ad62899-d2e2-442e-b14b-2a0acda6fa68-2017_Ducati_H.jpg
ebb87dfd-4f73-4a5d-95ba-90ab300512e2-ducati-939-hy.jpg
9a534088-0b59-49c9-a1e4-918fb90eae42-2017-ducati-m.jpg
790d4035-fb4d-4ec6-bf27-524716b35ac6-2000000009.jp.jpg
cea039c5-271e-4989-abf1-6bc3595b0664-110716-2017-d.jpg
f21404ec-73b8-47b2-b70f-006d66242426-2017-Ducati-M.jpg
cea1015d-891e-4c24-a74f-7c8c3de05870-2000000002.jp.jpg
0f3bd603-b366-4a98-8179-6029fe230f6e-797_3_OK_mod..jpg
0665ae24-a3c4-4cad-9efa-21b4e052e6c1-05-MULTISTRAD.jpg
a623fdd0-7ba6-4b59-832a-654ba83b6571-2017_Ducati_M.jpg
3f7c2e19-04ba-4b38-b973-7a17378c4ff6-multistrada_9.jpg
502106be-a56f-4792-8932-c5ca03b5f4a9-2000000001.jp.jpg
659037ab-b2fd-407f-be2b-7e4aba5d821e-img_KnzzwSiBR.jpg
1310f111-6

e9bec754-02fa-4904-8ef7-dcb0cb67deb2-2000000003.jp.jpg
6096511f-7e5d-4acb-a1b4-30a93e778bf5-Harley-Davids.jpg
44eeebc2-bb68-4c65-9aae-8e548c84bf05-harley-davids.jpg
30799417-4d86-42e8-a595-8dcf68c3fbe9-h-d-street-75.jpg
850bc609-324c-4302-9025-543b87846480-h-d-street-75.jpg
586aa76e-53dd-451f-b893-24bc860a4843-Harley-Davids.jpg
cad88656-b955-41f6-9cd7-a3273ef43fb9-harley-davids.jpg
f45f7912-5370-4396-8f5c-129c56ee44cf-maxresdefault.jpg
57b03d30-cf6a-4869-bfe4-de5cb817ae0a-Harley-Davids.jpg
b5bd906b-29c5-47c2-bfb1-f89ecc988d44-2017-Harley-D.jpg
2f14ae49-2d3f-4557-b76f-8be9ad49a23b-Harley-Davids.jpg
0f4dfea2-5c36-42e7-9311-2b7368bdae8b-Harley-Davids.jpg
0adc729a-3895-490c-a301-f25b49e4c1eb-2017-Harley-D.jpg
0db4ce9d-7670-4553-ad44-f368ec1e13ef-Harley-Davids.jpg
e9278977-e9fd-40bc-8888-632f7df23ff7-2017-harley-d.jpg
1dbaf941-c28f-44db-a81b-4c22765ecdb2-IMG_1638.jpg.jpg
ce9531ab-5bf9-4e8c-8716-95dec8df288d-Harley-Davids.jpg
7c9fb46b-4e3f-47d1-893e-dda167f597ba-2000000009.jp.jpg
3eaa315b-fe

590a7994-8b34-4a9e-9479-8b85a1260991-2017_honda_go.jpg
87cab0bb-1976-4993-b57a-48023b72d381-2017_honda_go.jpg
71ecaff0-1165-445a-adc1-5f6a69669fd6-2017-honda-gr.jpg
bbb6f7d0-3d06-4ca0-b59a-2b0fd67b7e40-YOSH-1007_Yos.jpg
695bdde7-3054-402a-80d5-e477fdfc989e-10075_Arrow-E.jpg
b46d5320-c311-48f6-b61c-fd0ce75a2841-2017_HONDA_GR.jpg
1b562b56-4445-4d0f-a525-03789f0939fb-new-2017-hond.jpg
116e49dd-c1c6-4e91-84ef-1de973891f9e-2017-honda-gr.jpg
230bdaf6-8f06-4344-9c97-e47b6923e966-new-2017-hond.jpg
db225980-954a-4dc6-be1c-ebde53f8945f-2016-honda-ms.jpg
548f597b-c04b-4d5d-8706-0532765c36d2-2017_Metropol.jpg
9760fdb6-a80a-4e1a-9afd-30c17d75ad06-2017-Honda-Me.jpg
0641e3d0-0ce6-4ac6-83ac-0fc0cc5e26e7-2000000002.jp.jpg
ab3ff7d3-bb0c-4850-941b-3fc043d95897-2017-Honda-Me.jpg
aba23bbc-2558-4ad6-bfb5-3dec1514d164-2017-Honda-Me.jpg
7adf9c36-d51c-40cc-b08a-fccda7407888-2017-Metropol.jpg
e92cb7d4-1727-4101-be77-93683fc79027-2000000003.jp.jpg
987d24fa-bd87-4833-a8fe-3bd0b660b405-Metropolitan_.jpg
25d01026-b

b9f642a5-33dc-4bff-a05e-e205898a667e-KLR650-First-.jpg
1d08d648-e06f-4542-a25e-ff59e2c1d68f-2017-Kawasaki.jpg
91fad9a7-056b-4d3c-902b-ad877c194ac7-maxresdefault.jpg
92780a2e-8e4b-4547-b1d4-212684916e6a-2017-Kawasaki.jpg
48243e1f-18df-41ec-a900-cb5f18514976-6000000002.jp.jpg
ade04490-cdb8-43f2-8bf8-fc83358d8e57-4260431-0-155.jpg
b7071636-ea9b-4ecd-a986-063183e5ac92-12FC7199-5C4D.jpg
c8ad3088-46f5-4b28-911f-f91707b80d31-2017-Kawasaki.jpg
8bf1e57f-d15e-4714-9e57-9ca2cc575b6c-KAWASAKI-KLX-.jpg
c5a7c0a4-d903-45be-b4f8-c51eeaaffc2a-6000000006.jp.jpg
ca069adf-1718-4ae8-9fe2-774693f99aab-6000000002.jp.jpg
f084642d-a2aa-44a2-9c6c-da74c3b3dec8-6000000005.jp.jpg
7c39fa5d-fa5d-40aa-8afe-5f4dd276b55c-6000000003.jp.jpg
a082992f-61df-43c0-ae9a-81a248842bf8-6000000001.jp.jpg
0afeceaa-c729-4d2b-88c1-e724dd1a95c6-maxresdefault.jpg
87615489-08a3-4dbf-82e4-c399a7bc2b12-2000000019.jp.jpg
95bb3c96-7326-4ffb-b70b-751820c7507e-qf0vzwng.ihn..ihn
39917850-db0b-4382-aa4d-137e3f989246-2000000001.jp.jpg
0ccb170f-2

2555f082-2125-445f-9cc8-c55ee04ce88f-kawasaki_vulc.JPG
d1e2e452-8323-4302-a3f1-0037b44fdeff-23EA53B5-1D80.jpg
b40c4e75-2d87-4998-91db-ef94c1dedf80-6000000001.jp.jpg
068764a1-baa8-4ea9-9927-7b396244bde2-6000000006.jp.jpg
c966ecdd-5701-4480-931c-8d6dbea34aab-2000000002.jp.jpg
d6ec9a05-3eaa-4a6e-abe6-9827ea4d3147-2000000003.jp.jpg
03b4a96a-0c26-48c6-8b4b-146a4e9b3fc6-5942-KAWASAKI.jpg
5b541246-cbc3-433b-bd12-63ce211b259d-2017-VULCAN-S.png
71e5fd86-c857-4f99-a360-2a3b6d9d88f2-Z1000-ABS-201.jpg
77f39c02-c4bc-4db7-adea-11568be50ea7-17ZR1000H_BK2.jpg
6a60d7bc-4fd6-4e1f-8aa7-7c94a45666f8-14802.jpg.jpg
44cc5ca6-0c5a-49c5-b340-f3ecc36e07d8-2017%2BKawasa.jpg
d326e7aa-1cb3-49aa-a080-fbebb642faa1-s-l1000.jpg.jpg
a850eb3b-21a9-4773-b653-6a3c317126f0-2017-Kawasaki.jpg
d2118080-42fb-47cf-842d-720b0b695d7d-wc1796389.jpg.jpg
9d087dc3-15ab-46f3-8613-d3cc3d89db68-2017-Kawasaki.jpg
b5c5ec69-8827-45b3-9423-18fc96a9bf27-2016-Kawasaki.jpg
0d8b8daa-8c83-4211-9e06-e3f1994bc132-2017-Kawasaki.jpg
ae86aa03-0730-40

2ddb0287-191b-483a-a04d-8e2fea775bf0-2017-suzuki-s.jpg
93400c7d-729f-40b6-b0b4-55234ab8847c-2017-suzuki-s.jpg
b6546204-f0f1-483d-b27f-0a299b9a092c-2017-SV650.jp.jpg
e5702afe-f858-4c27-b591-84503b3058a1-6000000002.jp.jpg
661900fd-a203-4486-96a3-bf9cb70f417c-Suzuki-SV650..png
fd19f25b-53fc-4a1b-b082-e5aac0fad6c6-2017-suzuki-s.jpg
16403d61-9a33-4e6e-b664-4417a6a42f10-2017-suzuki-s.jpg
198f290f-c614-49f2-b35e-a147b4b94304-6000000002.jp.jpg
df858f86-c8f9-4cb0-8cb4-51ed9c99410b-2017-Suzuki-S.jpg
250eea5d-2b95-4aed-a1f2-76a1b898f271-2017-suzuki-s.jpg
8402eda4-2979-459a-b5e9-32644ab3efcb-051816-2017-s.jpg
cdbaf9c3-94f1-4674-9871-3470c4526c8a-2017-Suzuki-S.jpg
1801849f-6fdf-45c7-b371-13300a0ff4dc-escape_suzuki.jpg
ee11a396-abe1-4252-a546-66a41b6c822f-2017-suzuki-s.jpg
3383f5a1-3414-4728-8ceb-56b500afb763-2017-Suzuki-S.jpg
a445a62d-ef00-4024-8ee3-4a0e5b15a62c-2017-suzuki-s.jpg
03ccc8da-634f-4954-80e8-a9d28f4f69eb-C73FABD6-16AF.jpg
0a7a6485-73f4-4efb-94fb-205710eeb79f-2017-Suzuki-T.jpg
45ac3d3d-1

9968726c-e437-4738-89f7-230d4fd05c58-2017-Yamaha-T.jpg
fc9c15fc-fb69-4e26-83d5-cc00f973008a-6000000001.jp.jpg
6b5de511-7b81-4c31-bc7b-299e133a7802-2017-Yamaha-T.jpg
004e76a2-9fb5-4825-b2e5-2298a648119d-6000000001.jp.jpg
2023e831-a98b-449d-8c87-84dcd1c7d660-2017_TT-R_230.png
206f1ca8-aac4-4223-8e3a-3bf8f4cae41e-2017-Yamaha-T.jpg
0cff3ee9-f385-4f95-b5fc-e19c44389e71-2018-Yamaha-T.jpg
77e82ef3-bd81-42ed-9d89-9775f57b2a11-2000000004.jp.jpg
aa515652-ebb9-490d-9de9-72e1e8b592da-2017-Yamaha-T.jpg
463e1571-5bab-4caf-bd3f-1f483c1a317c-2018-Yamaha-T.jpg
9f7666f7-67f0-4d95-9f0f-de773ea26e7d-2000000003.jp.jpg
fbc3be70-43b7-43a9-b04a-f8da21c156f6-yamaha-tt-r50.jpg
d45992f5-bd4f-41d2-875c-4d19ab7a25a4-2017-yamaha-t.jpg
350a6554-5a4b-4018-a05c-7c757cd8ad91-img_bimH7B0k5.jpg
858c84a6-585d-451a-b324-ea398746ab8d-yamaha-vino-c.jpg
7ddfaa49-d821-4177-a510-e41735226c96-2000000005.jp.jpg
b07458e0-e408-4f67-b479-64edfa10b349-yamaha-vino-c.jpg
130e0af9-1c12-4651-9e78-a2ebfb25fb26-yamaha-vino-c.jpg
20c93917-8

73147aa4-1bd0-4b82-b07d-d3ea538c6459-2018-Indian-C.jpg
81cb1095-a523-4f9f-9cd7-cd0fc889ed7b-2018-Indian-C.jpg
3abbe139-9432-4b99-929a-b067ee178b9d-2018-Indian-C.jpg
aa4bc6db-dea0-4f5a-a30b-e779f1a68d3e-2018-Indian-C.jpg
6dea6e6d-eb81-4d04-af17-28f3817865f0-2018-Indian-C.jpg
610fdf3f-a27c-496f-98fd-d37b43d58a9d-2018-Indian-C.jpg
e78e4caf-146f-4dcf-8765-68041a755a0d-2018-Indian-C.jpg
388f0b20-c115-4679-afbf-3cf81dcf1a76-2018-Indian-C.jpg
fcd016a0-b75b-48ff-aab5-875fc1fcaf0d-2000000002.jp.jpg
87f8d803-e875-4852-99c7-b9e79ddbf509-2018-Indian-C.jpg
e7e2469d-6da2-4cdd-9474-c3e5608564b9-2018-Indian-C.jpg
a7f7d356-962b-42ba-841a-d33760e49b40-2000000003.jp.jpg
a75325e7-d5e2-4603-91c2-349e5f8dd7d6-2000000003.jp.jpg
165d3b10-b9ac-41eb-8e86-bee59ed09125-2018-Indian-C.jpg
8c9e575b-4e17-4cee-99ee-dc90e5a82b7c-6000000011.jp.jpg
5f1872b2-e632-402b-a646-8d13ccac9707-072617-2018-i.jpg
1e4ba5a4-353b-4064-bcec-a804f353a676-2018-Indian-C.jpg
983e57bc-d4fc-4e1b-a3ca-2da5edf67327-2000000004.jp.jpg
ad1c2a32-6

ce3bbfa0-6a79-4b88-92fa-e56de913e146-006.jpg.jpg
79457a6b-d5c2-4d0d-b927-6afe8945e99c-6000000008.jp.jpg
c5ff180f-4e1e-42df-9104-b5e7c9037d4d-2018-Triumph-.jpg
4d0f34bd-ad26-4086-a1f8-c12144ec20fd-2018-Triumph-.jpg
67bb93c5-51ba-4008-938e-891b737b1bb1-6000000011.jp.jpg
d74c1ef2-a2cd-4373-841f-1cbca30a24bc-6000000010.jp.jpg
d3c27fba-d86a-4acc-b641-de3e4ca0134d-6000000009.jp.jpg
2eb7a139-dda5-44de-ba91-f825fcaec9d6-maxresdefault.jpg
de2aa8ad-5557-4623-ad4b-a188bd27730c-2018-Triumph-.jpg
0d048f1b-13e3-424a-928e-c79b53453a63-2018-Triumph-.jpg
118292e0-21d8-4a01-8446-7caa3ea9d97c-2018-Triumph-.jpg
4237fe5f-0882-4be0-866d-e1e56516cd53-triumph-bonne.jpg
f379e8be-17ac-409f-a470-a994f7e38ed6-triumph-bonne.jpg
c24e6d16-14e9-414e-84b0-87ec8741521b-6000000006.jp.jpg
cd335fdf-e2e2-4924-b7ec-da4e714c28b0-6000000001.jp.jpg
30608c7c-963a-4ab5-93a1-52252d28e8d7-6000000003.jp.jpg
3cca8a15-e1b3-4450-818d-b8ac3abba0b5-triumph-bonne.jpg
6dcefc2e-db52-4c37-9bb6-1217e707bafe-T16-L12-m_m_y.jpg
30a747ba-95fc-46

# Make a dataset with fewer than 8 images in the class

In [26]:
seven_max_data = get_maximum_images(raw_counts, 7)
# Make sure we still have at least three images
three_min_data = get_minimum_images(seven_max_data, 3)
segments7 = segment_images(three_min_data, test_proportion=0.2, val_proportion=0.2)

copy_clean_data(segments7, 'seven_max')


7a02acd0-7dc1-4ae4-9c82-96a1226a20db-2016-Ducati-X.jpg
44a7b5bc-1ba7-4e99-99a4-b5a7fad915c0-2016-Ducati-X.jpg
19859f2d-645d-445f-bcb3-2be3bb3379ec-2016-Ducati-X.jpg
03592c66-c564-41ac-a864-7fe6f828fef3-2016_CB300F_5.jpg
0ee47c8a-deac-4b56-bab7-806d0a7a535d-2016-honda-cb.jpg
77a591ac-fbb1-4f47-9ed6-aacb8eeef88e-2016-honda-cb.jpg
453dda26-a053-41e9-953c-12bc41ee150f-2016-honda-cr.jpg
cc057a1e-7097-4a33-9804-ca5388cda0f7-020416-2016-h.jpg
be59f994-d9ee-48f4-b8d4-b0640045575a-Honda-CRF-250.jpg
75fb3ab9-a7c8-48c2-9b1d-f258c00e3c99-honda%2Bnc700.jpg
2c39f815-dfeb-42d2-a5dd-bd15e342cf9d-2016-Honda-NC.jpg
e2bab9aa-ba33-4258-948f-4cd57c503f13-honda-nc750x-.jpg
3bda7ced-a45b-4767-8117-8e98ad934a48-E4C66354-1F9B.jpg
e416e9e3-8047-4e23-b29e-6d46ac6405de-2016-honda-xr.jpg
ba383600-6359-4992-8e59-9714b8f84f68-2016-honda-xr.jpg
773dea60-4c28-4065-84ce-7f36268580b4-UH200A_L6_YKV.jpg
ab2a5fdb-92c1-492e-b772-d089ed7b259c-2016-Suzuki-B.jpg
3abe16fa-01e8-4135-acb8-7378c0c3daf5-7.jpg.jpg
d2689cd3-c921-4c77

0d019f56-e3b6-46f3-81b4-0524afbaf4e9-2017-Kawasaki.jpg
62c4a03a-7298-4399-9c79-f12ae199b929-img_8mhAZaJT5.jpg
4d4cd09e-b891-4acb-b961-966479a06f32-yoshimura_r77.jpg
62836a36-2c35-4b51-815f-16cb1591612d-2017-Kawasaki.jpg
e517e00b-06d7-42dd-bfc5-65e8c130124d-2017-Kawasaki.jpg
4c871e94-9d49-4a90-a458-5200c710c9b0-2017_kawasaki.jpg
370b94c3-cc6d-48e3-9448-3ee034a6dc90-2017-Kawasaki.jpg
37478240-27b6-4901-8eca-c92e1c801985-06830DF5-6E0F.jpg
e7a170c4-6d43-4011-bc96-34a4801e8c61-6000000001.jp.jpg
2bc1deb1-5aad-4974-8766-fe0d1ac3df6b-KAWASAKI-ZX-1.jpg
ee0b07b2-39a7-40b0-9376-578f812e4f48-DSC04173.jpg.jpg
a153db3b-1531-4d59-ae88-6e061e5e6119-DSC04184-1024.jpg
72e214f1-1c0d-4f86-a7d9-bd7acd7440d1-ruiz170306042.jpg
2c3aff10-d269-4c09-9ab9-6edffa27fa68-2017-Moto-Guz.jpg
9ce97e22-672d-40a0-b6f6-1687ca925cbe-1.jpg.jpg
5d5250f1-2f67-4c02-ad42-cb06c0bdd46d-2.jpg.jpg
cd2eecb0-6196-4ec1-af7a-4ffb4df6de71-IMG_9118.jpg.jpg
aa8f70b5-6f87-4c66-b8db-bba23543200b-2017-Norton-C.jpg
073409de-91f7-495f-b088-956c

3279667b-c4f8-4c4b-9f2c-96cca3433ee5-2000000007.jp.jpg
3039931d-0f79-4a20-86d8-fabc2cf43092-2019-Indian-C.jpg
d428cd88-36b4-4461-a89f-892b25d58616-2000000041.jp.jpg
1961eeb9-3a1d-4a52-8713-24a5cbeaedd1-2019-Triumph-.jpg
ee9d8f63-8e48-45f0-9a98-4ea70ac6d58e-2019-Triumph-.jpg


125